In [1]:
import os
import xml.etree.cElementTree as ET
import re

In [2]:
files = os.listdir("./coropus_for_korzun/")
files.sort()
Time = '{http://www.abbyy.com/ns/Time#}'
rdfs = '{http://www.w3.org/2000/01/rdf-schema#}'
Aux = '{http://www.abbyy.com/ns/Aux#}'
regFile = open('./regex2','r')
regex = regFile.read()
output = open('./baselineoutput.txt','w')

In [3]:
TP = 0
FP = 0
FN = 0

for fl in files:      
    XML_FILE = os.path.join("./coropus_for_korzun/", fl)
    tree = ET.ElementTree(file = XML_FILE)
    root = tree.getroot()
    dates = set()
    for child in root:
        if child.tag == Time + 'PointOfTime':
            children = child.getchildren()        
            flag = False;       
            for part in children:            
                if part.tag == Time + 'month' or part.tag == Time + 'day' or part.tag == Time + 'year':
                    flag = True
                    break
            if flag == True:
                label = child.find(rdfs+'label')
                if label != None:
                    dates.add(label.text) 
                else:
                    dates.add(children[0].text)
      
    text = root.find(Aux+'TextAnnotations').find(Aux+'document_text').text
    answers = re.findall(regex, text)
    for i in range(len(answers)):
        if answers[i][0] == ' ':
            answers[i] = answers[i][1:]       
    answer_set = set(answers)
    
    output.write(fl)
    output.write('\n')
    for item in dates:
        try:
            output.write(item)
            output.write('; ')
        except UnicodeEncodeError as e:       
            print "Unicode Error in file: ", fl 
            
    output.write('\n')     
    for item in answer_set:
        output.write(item)
        output.write('; ')
    output.write('\n')   
    output.write('\n')
    
    TP += len(dates & answer_set)
    FP += len(answer_set - dates)
    FN += len(dates - answer_set)
    
regFile.close()
output.close()

Unicode Error in file:  806312newsML.xml


In [4]:
print TP, FN, FP

2529 772 804


In [5]:
precision = float(TP)/float(TP+FP)
recall = float(TP)/float(TP+FN)
F = 2*precision*recall/(precision+recall)

print "Precision: ", precision
print "Recall: ", recall
print "F: ", F

Precision:  0.758775877588
Recall:  0.766131475311
F:  0.762435936087
